In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen, urlparse
import re
import time
import requests

In [2]:
url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=end_date&seed=2743718&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [3]:
def engine(pages) : 
    url = url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=enddate&seed=2743718&page='
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    lst = []
    for i in range(1,pages) :
        r = requests.get(url+str(i),headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        lst.append(soup)
        i =+ 1   
    return lst

In [4]:
def scraper_prototype(text_profile, prefix):

    if '"name"' in text_profile:
        text_profile = re.sub('"', '', text_profile)

    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part) > 0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2, text_profile).group(1).rstrip(',')
                 for xxy1, xxy2 in zip(itemname[:-1], itemname[1:])]
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item, ans in zip(itemnames, ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict

In [5]:
def to_string(container) : 
    whole_string = str(container).replace('&amp', '').replace('&quot;', '')
    return whole_string

In [6]:
def get_name(whole_string) : 
    if '"name"' in whole_string:
        project_info = '"name"' + re.findall('"name"(.+?)"creator"', whole_string)[0]
    else:
        project_info = 'name' + re.findall('name(.+?)creator', whole_string)[0]
    project_info_dict = scraper_prototype(project_info, 'project')
    df1 = pd.DataFrame.from_dict(project_info_dict, orient='index').transpose()
    return df1

In [7]:
def get_creator(whole_string) : 
    if '"creator"' in whole_string:
        creator_string = re.findall(r'\"creator\"\:\{(.+?)\}', whole_string)[0]
    else:
        creator_string = re.findall(r'creator\:\{(.+?)\}', whole_string)[0]
    creator_info_dict = scraper_prototype(creator_string, 'creator')
    creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')
    df2 = pd.DataFrame.from_dict(creator_info_dict, orient='index').transpose()
    
    return df2

In [8]:
def get_location(whole_string) : 
    if '"location"' in whole_string:
        location_string = re.findall(r'\"location\"\:\{(.+?)\}', whole_string)[0]
    else:
        location_string = re.findall(r'location\:\{(.+?)\}', whole_string)[0]
    location_info_dict = scraper_prototype(location_string, 'location')
    df3 = pd.DataFrame.from_dict(location_info_dict, orient='index').transpose()
    return df3

In [9]:
def make_a_row(df1,df2,df3) : 
    df = pd.concat((df1, df2, df3), axis=1)
    return df

In [10]:
def row_flow(container) : 
    string = to_string(container)
    name = get_name(string)
    creator = get_creator(string)
    location = get_creator(string)
    #profile = get_profile(string)
    row = make_a_row(name,creator,location)
    return row

In [11]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [12]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
if r.status_code == 200 : 
    print('Reading the Website')
else : 
    print('Something went wrong')

Reading the Website


In [14]:
containers = soup.find_all('div', class_="js-react-proj-card")

In [15]:
row = row_flow(containers[0])
row0 = row_flow(containers[1])
row1 = row_flow(containers[2])
row2 = row_flow(containers[3])
row3 = row_flow(containers[4])
row4 = row_flow(containers[5])
row5 = row_flow(containers[6])
row6 = row_flow(containers[7])
row7 = row_flow(containers[8])
row8 = row_flow(containers[9])
row9 = row_flow(containers[10])
row10 = row_flow(containers[11])

In [35]:
df = pd.concat([row,row0,row2,row3,row4,row6,row7,row9,row10],ignore_index=True)

In [36]:
df = df.loc[:,~df.columns.duplicated()]

In [38]:
df.drop(['project_disable_communication','project_country_displayable_name',
         'project_currency_symbol','project_current_currency',
         'project_currency_trailing_code','project_staff_pick','project_backers_count',
         'project_static_usd_rate','project_converted_pledged_amount','project_fx_rate',
         'project_usd_exchange_rate','creator_is_registered','creator_is_email_verified',
         'creator_chosen_currency','creator_is_superbacker'],axis=1,inplace=True)

In [42]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_country,project_currency,project_deadline,project_state_changed_at,project_created_at,project_launched_at,project_is_starrable,project_usd_pledged,creator_id,creator_name,creator_slug,creator_avatar,creator_small
0,Sea Witch Collection,A beautiful set of pins full of nautical magic,450.0,16350.0,live,sea-witch-collection,US,USD,1646316017,1643724019,1638976932,1643724017,false,16350.0,2100073,Lily Brook Lane,lilybrooklane,{thumb:https://ksr-ugc.imgix.net/assets/033/17...,https://ksr-ugc.imgix.net/assets/033/174/595/2...
1,GHOSTS AND GHOULS AND OTHER CREEPY THINGS,Spine-tingling tales from NeoParadoxa featurin...,1500.0,5282.0,live,ghosts-and-ghouls-and-other-creepy-things,US,USD,1646317067,1643725071,1636655045,1643725067,false,5282.0,2096699057,eSpec Books,e-specbooks,{thumb:https://ksr-ugc.imgix.net/assets/009/34...,https://ksr-ugc.imgix.net/assets/009/342/575/c...
2,ZOOT! : The Card Game for Word Nerds,ZOOT! is played with a deck of 118 colourful l...,5000.0,5541.29,live,zoot-the-card-game-for-word-nerds,CA,CAD,1646318100,1643726101,1604093595,1643726100,false,4350.4090387582,1659177433,Joan Krygsman,zootzoot,{thumb:https://ksr-ugc.imgix.net/assets/036/14...,https://ksr-ugc.imgix.net/assets/036/149/328/c...
3,Customizable Fantasy World and Region Maps for...,More than 40 world ; regional fantasy maps of ...,750.0,1844.29,live,customizable-fantasy-world-and-region-maps-for...,CA,CAD,1646318805,1645109206,1642576264,1645109205,false,1450.0149726937,1173384431,Jean Francois Senay,jfsenay,{thumb:https://ksr-ugc.imgix.net/assets/009/81...,https://ksr-ugc.imgix.net/assets/009/813/811/4...
4,Super Best Friend #1-2: A Superhero Adventure ...,The live-streamer sidekick of the world’s grea...,7500.0,15132.0,live,super-best-friend-1-2-a-superhero-adventure-comic,US,USD,1646319375,1643727377,1629226778,1643727375,false,15132.0,827962922,Jason Inman,jawiin,{thumb:https://ksr-ugc.imgix.net/assets/006/64...,https://ksr-ugc.imgix.net/assets/006/648/968/7...
5,City of Spiritdale - A Fantasy City,"3D Printable STL Files, Terrain for Tabletop, ...",6300.0,142419.29,live,city-of-spiritdale-a-fantasy-city,CA,CAD,1646319622,1643727624,1641391755,1643727622,false,111811.9005699982,2145431169,TabletopTerrain,ian-margherit,{thumb:https://ksr-ugc.imgix.net/assets/035/23...,https://ksr-ugc.imgix.net/assets/035/236/461/6...
6,5e RPG - Tomb of the Colossus Gods - 30+ Battl...,Tomb of the Colossus Gods is a Dark Epic Fanta...,12000.0,53928.6,live,tomb-of-the-colossus-gods-campaign-and-setting...,ES,EUR,1646319636,1643727638,1622879616,1643727636,false,60193.745397852,809411887,Lion Banner Games,lionbanner,{thumb:https://ksr-ugc.imgix.net/assets/030/99...,https://ksr-ugc.imgix.net/assets/030/999/433/e...
7,The Magic Forest Tarot Deck,A magical tarot deck filled with gnomes and ot...,1000.0,1611.0,live,the-magic-forest-tarot-deck,US,USD,1646319675,1643727676,1634782240,1643727675,false,1611.0,199594465,Cassandra Helton,magicforesttarot,{thumb:https://ksr-ugc.imgix.net/assets/035/35...,https://ksr-ugc.imgix.net/assets/035/355/443/5...
8,Mossvale,Welcome to the wonderful world of Mossvale wit...,4400.0,4816.0,live,mossvale,US,USD,1646319967,1643727968,1642607272,1643727967,false,4816.0,258235766,Ageless Comics Publishing,dinothrashers2,{thumb:https://ksr-ugc.imgix.net/assets/030/03...,https://ksr-ugc.imgix.net/assets/030/034/197/9...


In [84]:
lst =[]
my_dict ={}
for i in range(0,len(df)) :
    creator_id = str(df.loc[i]['creator_id'])
    html = requests.get('https://www.kickstarter.com/profile/'+creator_id+'/about', headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        #print(str(df.loc[i]['creator_id']))
        lst.append((a['href'], creator_id ))
        my_dict[creator_id]=(a['href'])

In [91]:
lst2 = []

In [92]:
for el in lst : 
    if is_valid(el[0]) != True :
        lst.remove(el)
    if el[0].find('http') != -1 :
        lst2.append(el)
        

In [93]:
lst2

[('https://www.instagram.com/lilybrooklane', '2100073'),
 ('https://www.facebook.com/E-Spec-Books-768052066586116/', '2096699057'),
 ('https://twitter.com/eSpecBooks', '2096699057'),
 ('https://especbooks.wordpress.com/', '2096699057'),
 ('https://www.patreon.com/especbooks', '2096699057'),
 ('http://especbooks.square.site/', '2096699057'),
 ('https://zootzoot.ca', '1659177433'),
 ('http://www.dungeoncreatorsets.com', '1173384431'),
 ('http://www.smalldungeonmaps.com', '1173384431'),
 ('http://www.twitter.com/Jawiin', '827962922'),
 ('http://www.geekhistorylesson.com', '827962922'),
 ('http://www.youtube.com/Jawiin', '827962922'),
 ('http://www.jasoninman.com', '827962922'),
 ('https://www.instagram.com/jawiin/', '827962922'),
 ('https://www.lionbanner.com', '809411887'),
 ('https://www.instagram.com/cassie_h_art/', '199594465'),
 ('https://www.facebook.com/cassieheltonart', '199594465')]

In [94]:
s = []
for el in  lst2: 
    try :
        r = requests.get(el[0])
        t = str(r.text)
        shopify = t.find('shopify')
        s.append((shopify,el))
    except ConnectionError :
        pass

In [95]:
s

[(-1, ('https://www.instagram.com/lilybrooklane', '2100073')),
 (-1,
  ('https://www.facebook.com/E-Spec-Books-768052066586116/', '2096699057')),
 (-1, ('https://twitter.com/eSpecBooks', '2096699057')),
 (-1, ('https://especbooks.wordpress.com/', '2096699057')),
 (-1, ('https://www.patreon.com/especbooks', '2096699057')),
 (-1, ('http://especbooks.square.site/', '2096699057')),
 (-1, ('https://zootzoot.ca', '1659177433')),
 (-1, ('http://www.dungeoncreatorsets.com', '1173384431')),
 (-1, ('http://www.smalldungeonmaps.com', '1173384431')),
 (-1, ('http://www.twitter.com/Jawiin', '827962922')),
 (-1, ('http://www.geekhistorylesson.com', '827962922')),
 (-1, ('http://www.youtube.com/Jawiin', '827962922')),
 (-1, ('http://www.jasoninman.com', '827962922')),
 (-1, ('https://www.instagram.com/jawiin/', '827962922')),
 (-1, ('https://www.lionbanner.com', '809411887')),
 (-1, ('https://www.instagram.com/cassie_h_art/', '199594465')),
 (-1, ('https://www.facebook.com/cassieheltonart', '19959446

In [97]:
type(s[0])

tuple

In [ ]:
las_lst = []
for el in s : 
    if el[0]  == -1 :
        s.remove(el)
    else : 
        last_lst.append(el[1])

In [ ]:
if len(last_lst) == 0 : 
    print('No shopify user for now')
else : 
    print(len(last_lst),' Shopify user detected')